In [30]:
import pandas as pd
import numpy as np
# 정규화 / 표준화
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [31]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DL/ozone.csv')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Ozone    116 non-null    float64
 1   Solar.R  146 non-null    float64
 2   Wind     153 non-null    float64
 3   Temp     153 non-null    int64  
 4   Month    153 non-null    int64  
 5   Day      153 non-null    int64  
dtypes: float64(3), int64(3)
memory usage: 7.3 KB


In [33]:
df.head()

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
4,NaN,NaN,14.3,56,5,5


In [34]:
df = df.loc[:, ['Ozone','Temp']]
# 편의를 위해 날림 -> 반드시 결측치 재검토 요!
df.dropna(inplace=True)
X = df['Temp']
y = df['Ozone']


In [35]:
X.shape

(116,)

In [36]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Ozone,116.0,42.12931,32.987885,1.0,18.0,31.5,63.25,168.0
Temp,116.0,77.87069,9.485486,57.0,71.0,79.0,85.00,97.0


In [37]:
X.head()

0    67
1    72
2    74
3    62
5    66
Name: Temp, dtype: int64

In [38]:
y.head()

0    41.0
1    36.0
2    12.0
3    18.0
5    28.0
Name: Ozone, dtype: float64

In [39]:
X = X.values.reshape(-1,1)
y = y.values.reshape(-1,1)

In [40]:
sc = MinMaxScaler()
sc.fit(X)
sc.fit(y)
X_sc = sc.transform(X)
y_sc = sc.transform(y)

In [41]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense
# 확률적 경사하강법 SGD
from keras.optimizers import SGD, Adam

In [49]:
# 입력계층 ~ 출력계층 박스 - 내부적 박스 생성
model = Sequential()
# 입력 계층이 1개인 - 평탄화 작업
model.add(Flatten(input_shape=(1,)))
# 출력층 생성
model.add(Dense(units=1, activation='linear'))
# 컴파일 - 최적화 함수(optimizer) // cf) 0.01 == 1e-2
# 손실(loss) 함수 - mean squared error
model.compile(optimizer=Adam(learning_rate=1e-2),
              loss='binary_crossentropy')

In [ ]:
# epochs -> 1000번 반복 / verbose -> 작업 진행도 보여줌
model.fit(X_sc,y_sc, epochs=1000)

In [43]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 1)                 0         
                                                                 
 dense_1 (Dense)             (None, 1)                 2         
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [50]:
x_pred = np.array([[70]])
# fit 사용하면 안됨. 재학습 조심
y_pred = model.predict(sc.transform(x_pred))
# 정규화를 다시 값으로 변환
sc.inverse_transform(y_pred)

1/1 [==============================] - 0s 46ms/step


array([[-70.31054]], dtype=float32)

#### Adam - 정규화 X -> array([[39.019352]], dtype=float32)
#### Adam - 정규화 O -> array([[23.675364]], dtype=float32)


In [45]:
score = model.evaluate(X_sc,y_sc)
score

4/4 [==============================] - 0s 4ms/step - loss: 88.9178


88.9178237915039